In [1]:
from pyiem.util import get_dbconn
from shapely.wkt import loads

POSTGIS = get_dbconn('postgis')
cursor = POSTGIS.cursor()

roads = []
freq = []

cursor.execute("""
WITH obs as (
 SELECT segid, sum(lead - valid) from
 (SELECT segid, valid,  lead(valid) OVER (PARTITION by segid ORDER by valid ASC),
 cond_code from roads_log WHERE valid > '2020-10-01' and valid < '2021-01-20') as foo
 WHERE cond_code > 0 GROUP by segid
)
 SELECT obs.sum, ST_AsEWKT(ST_Transform(base.simple_geom,4326)) from obs JOIN
 roads_base base on (obs.segid = base.segid) 

""")

for row in cursor:
    if row[1] is None:
        continue
    roads.append(loads(row[1][10:]))
    freq.append(row[0])
print(len(freq))

1005


In [2]:
import datetime

sts = datetime.datetime(2020,10,18)
ets = datetime.datetime(2021,1,20)
period = (ets - sts).total_seconds()

In [12]:
from pyiem.plot import MapPlot, whitebluegreenyellowred
import numpy as np
import matplotlib.colors as mpcolors
import cartopy.crs as ccrs

cmap = whitebluegreenyellowred()
clevs = np.arange(0,13,2)
norm = mpcolors.BoundaryNorm(clevs, cmap.N)

m = MapPlot(title='18 Oct 2020 - 20 Jan 2021 Percent Time with Non-Seasonable Driving Condition',
            subtitle='Based on Iowa DOT Road Condition Reports, till 10 PM 19 Jan 2021',
            axisbg='white',
            twitter=True)

for geoms, f in zip(roads,freq):
    if f is None:
        continue
    dur = f.days * 86400. + f.seconds
    percent = dur / period * 100.
    for geo in geoms:
        x,y = geo.xy
        #points = m.ax.projection.transform_points(ccrs.PlateCarree(), np.array(x), np.array(y))
        m.ax.plot(x, y, lw=3., color=cmap( norm([percent,]) )[0], transform=ccrs.PlateCarree(), zorder=10)

m.draw_colorbar(clevs, cmap, norm, units='percent')
m.drawcities()
        
m.postprocess(filename='210120.png')
m.close()